# Import Libraries and Datasets

In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [6]:
# Load paths for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project/'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

for split in splits:
    print(f"\n{split.capitalize()} set:")
    for category in labels:
        path = os.path.join(images_path, split, category)
        print(f"{category.capitalize()}: {len(os.listdir(path))} images")

train_ratio, test_ratio, val_ratio = 0.7, 0.15, 0.15
total_images = 22710 + 20140
print(f"\nExpected Train: {int(total_images * train_ratio)}")
print(f"Expected Test: {int(total_images * test_ratio)}")
print(f"Expected Validation: {int(total_images * val_ratio)}")


Train set:
Wildfire: 15750 images
Nowildfire: 14500 images

Test set:
Wildfire: 3480 images
Nowildfire: 2820 images

Valid set:
Wildfire: 3480 images
Nowildfire: 2820 images

Expected Train: 29994
Expected Test: 6427
Expected Validation: 6427


# Data overview about datasets statistcs

# Data Visualization

# Image Quality Analysis

# Data Augmentation

# Feature Engineering